# How to use
Run all and enter

In [1]:
from torch.cuda import graph

# Get documents
documents = []

for i in range(int(input("Enter the number of documents for the AI to understand"))):
    documents.append(input("Copy the document here").replace("\n\n", ""))

documents

['The Semantic Web, sometimes known as Web 3.0 (not to be confused with Web3), is an extension of the World Wide Web through standards[1] set by the World Wide Web Consortium (W3C). The goal of the Semantic Web is to make Internet data machine-readable.To enable the encoding of semantics with the data, technologies such as Resource Description Framework (RDF)[2] and Web Ontology Language (OWL)[3] are used. These technologies are used to formally represent metadata. For example, ontology can describe concepts, relationships between entities, and categories of things. These embedded semantics offer significant advantages such as reasoning over data and operating with heterogeneous data sources.[4]These standards promote common data formats and exchange protocols on the Web, fundamentally the RDF. According to the W3C, "The Semantic Web provides a common framework that allows data to be shared and reused across application, enterprise, and community boundaries."[5] The Semantic Web is the

In [ ]:
question = input("Enter your question: ")

In [ ]:
from GraphGen.graph_gen_integration import GraphGen
# Parse documents
triples = []

graph_gen = GraphGen([], keyword_reward=1.1)

for idx, document in enumerate(documents):
    print(f"==================== Process document {idx+1}/{len(documents)} ====================")
    graph_gen.reset_model(triples, idx)
    triples += graph_gen.generate(document)
    
print("ACTUALLY DONE!")

In [3]:
triples

[Triple(subject='Semantic_Web', predicate='extension', object='World_Wide_Web_Consortium', id=0, document_id=0),
 Triple(subject='Semantic_Web', predicate='isPartOf', object='World_Wide_Web', id=0, document_id=0),
 Triple(subject='Semantic_Web', predicate='isPartOf', object='World_Wide_Web_Consortium', id=0, document_id=0),
 Triple(subject='Semantic_Web', predicate='isPartOf', object='Web3.0', id=0, document_id=0),
 Triple(subject='Semantic_Web', predicate='technology', object='RDF', id=2, document_id=0),
 Triple(subject='Semantic_Web', predicate='technology', object='Web_Ontology_Language', id=2, document_id=0),
 Triple(subject='Semantic_Web', predicate='category', object='Ontology', id=3, document_id=0),
 Triple(subject='Semantic_Web', predicate='isPartOf', object='World_Wide_Web_Consortium_W3C', id=5, document_id=0),
 Triple(subject='Semantic_Web', predicate='application', object='Library_and_information_science_research', id=8, document_id=0),
 Triple(subject='Semantic_Web', predic

In [4]:
from GraphGen import graph_gen_integration

graph_gen_integration.draw_graph(triples)

network.html


In [6]:
import torch, gc
graph_gen.free_model()
del graph_gen
gc.collect()
torch.cuda.empty_cache()

from QueryGen.query_gen_integration import QueryGen
query_gen = QueryGen(triples)

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [12]:
query = query_gen.generate(question)
query

KeyboardInterrupt: Interrupted by user

In [19]:
# query = 'SELECT DISTINCT ?uri WHERE { <Semantic_Web> <isPartOf> ?uri  . }'

In [20]:
from rdflib import Graph, Literal, RDF, URIRef
g = Graph()
for triple in triples:
    g.add((URIRef(triple.subject), URIRef(triple.predicate), URIRef(triple.object)))

"RDF_RDF_Graph_Statement_1.0.0" does not look like a valid URI, trying to serialize this will break.
"user-friendly" does not look like a valid URI, trying to serialize this will break.
"web_server" does not look like a valid URI, trying to serialize this will break.
"web_page" does not look like a valid URI, trying to serialize this will break.
"plain_text" does not look like a valid URI, trying to serialize this will break.
"images" does not look like a valid URI, trying to serialize this will break.
"embedded_video" does not look like a valid URI, trying to serialize this will break.
"embedded_audio" does not look like a valid URI, trying to serialize this will break.
"embedded_urls" does not look like a valid URI, trying to serialize this will break.
"HTTP" does not look like a valid URI, trying to serialize this will break.
"Internet_protocol" does not look like a valid URI, trying to serialize this will break.
"Internet_server" does not look like a valid URI, trying to serialize 

In [22]:
# Apply the query to the graph and iterate through results
r_r = []
for r in g.query(query):
    print(r[0])
    r_r.append(str(r[0]))
r_r

World_Wide_Web
World_Wide_Web_Consortium
Web3.0
World_Wide_Web_Consortium_W3C
Semantic_Web_Dream


['World_Wide_Web',
 'World_Wide_Web_Consortium',
 'Web3.0',
 'World_Wide_Web_Consortium_W3C',
 'Semantic_Web_Dream']

In [25]:
query_gen.free_model()
del query_gen
gc.collect()
torch.cuda.empty_cache()

In [26]:
from AnswerGen.answer_gen_integration import AnswerGen
answer_gen = AnswerGen()

answer = answer_gen.generate(question, str(r_r))
answer

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


'The parts of the Semantic Web are World_Wide_Web, World_Wide_Web_Consortium, Web3.0, World_Wide_Web_Consortium_W3C, Semantic_Web_Dream.<eos>'

In [27]:
answer_gen.free_model()
del answer_gen
gc.collect()
torch.cuda.empty_cache()

print(answer.replace("<eos", ""))

The parts of the Semantic Web are World_Wide_Web, World_Wide_Web_Consortium, Web3.0, World_Wide_Web_Consortium_W3C, Semantic_Web_Dream.>
